In [3]:
import os
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import types        

In [5]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('process-taxi-data') \
    .getOrCreate()

# Q1 spark version  
print(f"Spark Version - {spark.version}")

23/03/02 17:18:02 WARN Utils: Your hostname, ozkary-dev-2020 resolves to a loopback address: 127.0.1.1; using 192.168.214.247 instead (on interface eth0)
23/03/02 17:18:02 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/02 17:18:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/03/02 17:18:06 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
Spark Version - 3.3.2


In [10]:
# Read the parquet files into a data frame
data_path = 'data/fhvhv/'
df = spark.read.parquet(data_path)
                                                                                
# Q3: How many taxi trips were there on June 15 2021?
pickup_dt = '2021-06-15'
from pyspark.sql import functions as F
total = df.withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .filter(f"pickup_date = '{pickup_dt}'") \
    .count()

print(f" Trips in 06/15/2021 {total}")

 Trips in 06/15/2021 452470


In [11]:

df.createOrReplaceTempView('fhvhv_tripdata')
spark.sql(f"""
SELECT
    COUNT(1)
FROM 
    fhvhv_tripdata
WHERE
    to_date(pickup_datetime) = '{pickup_dt}';
""").show()

+--------+
|count(1)|
+--------+
|  452470|
+--------+



In [12]:
spark.sql("""
SELECT
    to_date(pickup_datetime) AS pickup_date,
    MAX((CAST(dropoff_datetime AS LONG) - CAST(pickup_datetime AS LONG)) / (60 * 60)) AS duration
FROM 
    fhvhv_tripdata
GROUP BY
    1
ORDER BY
    2 DESC
LIMIT 5;
""").show()

+-----------+------------------+
|pickup_date|          duration|
+-----------+------------------+
| 2021-06-25|  66.8788888888889|
| 2021-06-22|25.549722222222222|
| 2021-06-27|19.980833333333333|
| 2021-06-26|18.197222222222223|
| 2021-06-23|16.466944444444444|
+-----------+------------------+



In [14]:
zones_path = 'data/zones'
df_zones = spark.read.parquet(zones_path)
df_zones.columns
['LocationID', 'Borough', 'Zone', 'service_zone']

print('Most frequent pickup location zone')
df_zones.createOrReplaceTempView('zones_data')
spark.sql("""
SELECT
   pul.Zone,
   COUNT(1) as Total
FROM 
    fhvhv_tripdata fhv 
    INNER JOIN zones_data pul ON fhv.PULocationID = pul.LocationID    
GROUP BY 
    1
ORDER BY
    2 DESC
LIMIT 5;
""").show()

Most frequent pickup location zone


+-------------------+------+
|               Zone| Total|
+-------------------+------+
|Crown Heights North|231279|
|       East Village|221244|
|        JFK Airport|188867|
|     Bushwick South|187929|
|      East New York|186780|
+-------------------+------+

